https://huggingface.co/d4data/biomedical-ner-all
https://huggingface.co/datasets/singh-aditya/MACCROBAT_biomedical_ner

https://www.freecodecamp.org/news/getting-started-with-ner-models-using-huggingface/
https://medium.com/@minhle_0210/pos-tagging-medical-ner-ffcdaef7a7b3
https://github.com/dreji18/Bio-Epidemiology-NER

https://huggingface.co/distilbert/distilbert-base-uncased
https://huggingface.co/google-bert/bert-base-uncased
https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT
https://huggingface.co/Charangan/MedBERT

https://wandb.ai/jack-morris/david-vs-goliath/reports/Does-Model-Size-Matter-A-Comparison-of-BERT-and-DistilBERT--VmlldzoxMDUxNzU#:~:text=The%20BERT%20authors%20recommend%20fine,5e%2D5%2C%203e%2D5
https://datascience.stackexchange.com/questions/64583/what-are-the-good-parameter-ranges-for-bert-hyperparameters-while-finetuning-it

In [3]:
tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="max")  # device=0 (gpu)

text = """The patient reported no recurrence of palpitations at follow-up 6 months after the ablation."""

out = pipe(text)
print(out)

spans = []

for row in out:
    spans.append((row["start"], row["end"], row["entity_group"]))

# show_span_ascii_markup(text, spans)
show_span_box_markup(text, spans)

[{'entity_group': 'Sign_symptom', 'score': 0.9999311, 'word': 'palpitations', 'start': 38, 'end': 50}, {'entity_group': 'Clinical_event', 'score': 0.99975544, 'word': 'follow', 'start': 54, 'end': 60}, {'entity_group': 'Date', 'score': 0.999867, 'word': '6 months after', 'start': 64, 'end': 78}]


In [13]:
import warnings
import gc

import torch
from torch.utils.data import random_split

from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers.utils.logging import set_verbosity_error

from ipymarkup import show_span_ascii_markup, show_span_box_markup

from mrb_ner_config import *
from mrb_ner_dataset import MRBNERDataset
from mrb_ner_evaluator import MRBNEREvaluator

warnings.filterwarnings("ignore")

In [12]:
# get dataset from huggingface
data = load_dataset(DATA_PATH)

# extract NER labels
label_names = data["train"].features["ner_labels"].feature.names
id2label = {}
label2id = {}

for idx, label in enumerate(label_names):
    id2label[idx] = label
    label2id[label] = idx

# split dataset into train, val and test splits
data_splits = {}
data_splits["train"], data_splits["val"], data_splits["test"]  = random_split(data["train"], [0.7, 0.15, 0.15])

In [8]:
evaluator = MRBNEREvaluator(metric="seqeval", id2label=id2label)

for m in MODELS:
    save = {
        "save_trainer": None,
        "test_metrics": None,
        "param": None
    }
    
    for lr in LRATES: 
        print(f"\n---------------------- Training Model (Model:{m['name']}, Lr:{lr}) ----------------------")

        tokenizer = AutoTokenizer.from_pretrained(m["path"])
        data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
        model = AutoModelForTokenClassification.from_pretrained(
            pretrained_model_name_or_path=m["path"],
            label2id=label2id,
            id2label=id2label,
            ignore_mismatched_sizes=True,
            num_labels=len(label2id)
        )

        datasets = {}
        datasets["train"] = MRBNERDataset(data=data_splits["train"], tokenizer=tokenizer, id2label=id2label, label2id=label2id, max_len=512)
        datasets["val"] = MRBNERDataset(data=data_splits["val"], tokenizer=tokenizer, id2label=id2label, label2id=label2id, max_len=512)
        datasets["test"] = MRBNERDataset(data=data_splits["test"], tokenizer=tokenizer, id2label=id2label, label2id=label2id, max_len=512)
        
        training_args = TrainingArguments(
            output_dir=f"../output/{m['name']}/{lr}",
            overwrite_output_dir=True,
            per_device_train_batch_size=BATCH_SIZE,
            per_device_eval_batch_size=BATCH_SIZE,
            num_train_epochs=EPOCHS,
            learning_rate=lr,
            weight_decay=WDECAY,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=datasets["train"],
            eval_dataset=datasets["val"],
            data_collator=data_collator,
            tokenizer=tokenizer,
            compute_metrics=evaluator.compute_metrics
        )

        trainer.train()
        predictions, _, metrics = trainer.predict(datasets["test"])

        if save["test_metrics"] == None or save["test_metrics"]["test_overall_accuracy"] < metrics["test_overall_accuracy"]:
            best_accuracy = save['test_metrics']['test_overall_accuracy'] if save['test_metrics'] else None
            print(f"Updating best accuracy: {best_accuracy} -> {metrics['test_overall_accuracy']}")

            save["save_trainer"] = trainer
            save["test_metrics"] = metrics
            save["param"] = lr

    print(f"\nSaving best {m['name']} model...")
    for key, value in save["test_metrics"].items():
        print(f"{key}: {value}")

    save["save_trainer"].save_model(f"../output/{m['name']}/save_model")

    del save, tokenizer, data_collator, model, datasets, training_args, trainer, predictions, _, metrics
    gc.collect()
    torch.cuda.empty_cache()


---------------------- Training Model (Model:distilbert-base-uncased, Lr:5e-05) ----------------------


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 35/35 [00:05<00:00,  7.30it/s]

{'eval_loss': 2.166745662689209, 'eval_overall_precision': 0.031503579952267304, 'eval_overall_recall': 0.0234208658623137, 'eval_overall_f1': 0.026867494402605333, 'eval_overall_accuracy': 0.1462021062065875, 'eval_runtime': 0.5138, 'eval_samples_per_second': 58.383, 'eval_steps_per_second': 15.569, 'epoch': 1.0}


100%|██████████| 35/35 [00:06<00:00,  5.42it/s]


{'train_runtime': 6.4521, 'train_samples_per_second': 21.698, 'train_steps_per_second': 5.425, 'train_loss': 2.5291458129882813, 'epoch': 1.0}


...

{'eval_loss': 2.6164839267730713, 'eval_overall_precision': 0.0, 'eval_overall_recall': 0.0, 'eval_overall_f1': 0.0, 'eval_overall_accuracy': 0.00021800741225201656, 'eval_runtime': 0.858, 'eval_samples_per_second': 34.965, 'eval_steps_per_second': 9.324, 'epoch': 1.0}


100%|██████████| 35/35 [00:12<00:00,  2.90it/s]


{'train_runtime': 12.0831, 'train_samples_per_second': 11.586, 'train_steps_per_second': 2.897, 'train_loss': 3.1084603445870536, 'epoch': 1.0}


100%|██████████| 8/8 [00:00<00:00, 11.46it/s]



Saving best medbert model...
test_loss: 2.0795881748199463
test_overall_precision: 0.058941728064300064
test_overall_recall: 0.033846153846153845
test_overall_f1: 0.043000244319569995
test_overall_accuracy: 0.1315203375923104
test_runtime: 0.801
test_samples_per_second: 37.453
test_steps_per_second: 9.988


In [31]:
pipe = pipeline("ner", model=model, tokenizer=tokenizer, device=0, aggregation_strategy="max")  # device=0 (gpu)
text = """The patient reported no recurrence of palpitations at follow-up 6 months after the ablation."""

out = pipe(text)
print(out)

spans = []

for row in out:
    spans.append((row["start"], row["end"], row["entity_group"]))

show_span_box_markup(text, spans)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'SIGN_SYMPTOM', 'score': 0.49153978, 'word': 'recurrence', 'start': 24, 'end': 34}, {'entity_group': 'SIGN_SYMPTOM', 'score': 0.702181, 'word': 'palpitations', 'start': 38, 'end': 50}, {'entity_group': 'DATE', 'score': 0.46011654, 'word': '6 months after', 'start': 64, 'end': 78}]
